# Footprint TOA cloudmasks for use with SRLite
cloudmasks can be very detailed - so need to be written out as individual (TOA-specific) geodataframes during multiprocessing - then concatenated

In [1]:
import geopandas as gpd
import pandas as pd
import os, sys
sys.path.append('/home/pmontesa/code/geoscitools')
import footprintlib
import geoscilib

import matplotlib.pyplot as plt
import contextily as ctx
import re
import fiona
import math
import glob

/tmp/ipykernel_244901/2629257221.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [29]:
import importlib
import footprintlib
importlib.reload(footprintlib)

<module 'footprintlib' from '/panfs/ccds02/home/pmontesa/code/geoscitools/footprintlib.py'>

In [3]:
from multiprocessing import Pool
from functools import partial

### Dirs

In [13]:
MAINDIR = '/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1'
OUTDIR = '/explore/nobackup/people/pmontesa/srlite'

#TARGET = 'Alaska_ahri_20221004'
#TARGET_DIR_TOA = f'/explore/nobackup/projects/ilab/data/srlite/toa/{TARGET}'

TARGET = 'alaska/batch_*/4_band'
TARGET_DIR_TOA = f'{MAINDIR}/toa/{TARGET}'

# Cloudmask dir
TARGET_DIR_CLOUDMASK = f'{MAINDIR}/cloudmask/{TARGET}'

# Final SRLite output
TARGET_DIR_SR = f'{MAINDIR}/srlite/{TARGET}'

## SR: Build gdf

In [14]:
# Get footprints gdf from a target dir
TARGET_DIR_SR = TARGET_DIR_SR.rstrip('/')
print(TARGET_DIR_SR)

# Get the list of filenames
#SEARTH_STRING = '/*/*/5-toas/*toa.tif'
SEARTH_STRING = '/split_*/*.tif'

f_list_sr = glob.glob(f"{TARGET_DIR_SR}{SEARTH_STRING}", recursive=True)
print(f'Search returned {len(f_list_sr)} SRs')

/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/srlite/alaska/batch_*/4_band
Search returned 2075 SRs


# Footprint the cloudmask
To really know where valid data from SRlite exists, footprint the cloudmask

In [15]:
CLOUDDIR = os.path.join(MAINDIR, 'cloudmask')

In [17]:
if False:
    import rasterio as rio
    from rasterio.crs import CRS
    from rasterio.plot import show_hist, show
    vhrlib.map_image_band('/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask/split_10/WV02_20101003_M1BS_103001000772AA00-toa.cloudmask.tif', band_num=1, vmin=0, vmax=1)

In [18]:
# Get footprints gdf from a target dir
TARGET_CLOUDDIR = CLOUDDIR.rstrip('/')
print(TARGET_CLOUDDIR)

# Get the list of filenames
SEARTH_STRING = '/**/*cloudmask.tif'
f_list_cloudmask = glob.glob(f"{TARGET_CLOUDDIR}{SEARTH_STRING}", recursive=True)
print(f'Search returned {len(f_list_cloudmask)} cloudmasks')

/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask
Search returned 3109 cloudmasks


In [19]:
# # Make footprint gdfs of the the filenames, concat to single gdf
# f_gdf_lst = [footprintlib.raster_footprint(r_fn, DO_DATAMASK=True, GET_ONLY_DATASETMASK=True, MASK_OUT_VALUE=255, R_READ_MODE='r', MANY_CRS=True) for r_fn in f_list_cloudmask[0:1]]
# footprint_gdf_cloudmask = footprintlib.build_footprint_db(f_gdf_lst, TO_GCS=False, WRITE_GPKG=False, OUT_F_NAME='', OUT_LYR_NAME=TARGET, DROP_DUPLICATES=True)

## Return a GDF of the areas of good TOA pixels to help use SRlite
eg; we dont want to use SRlite for training in areas that are cloudmasked
resulting from a dilation of the cloudmask that will permit tiles of size N_ITER x 2

In [20]:
LAND_VAL = 0
CLOUD_VAL = 1
N_ITER=128
TARGET

'alaska/batch_*/4_band'

### FYI, get the number of unique strips from the paths of TOA cloudmasks

In [21]:
stripname_list = list(set([os.path.basename(f).split('-toa')[0] for f in f_list_cloudmask]))
print(f'{len(stripname_list)} unique TOA strips with cloudmasks')
stripname_list[0]

3100 unique TOA strips with cloudmasks


'WV02_20140522_M1BS_10300100302B0F00'

#### Get the list of paths to cloudmasks that correspond to strip names from the paths of SRLite
this will contain duplicate strips when they are in different subdirs.

In [22]:
f_list_cloudmask_final = [cloudmask for cloudmask in f_list_cloudmask for stripname in stripname_list if stripname in cloudmask]
print(f'{len(f_list_cloudmask_final)} total TOA strips with cloudmasks')
len(set(f_list_cloudmask_final))

3109 total TOA strips with cloudmasks


3109

In [23]:
# Find the duplicates
import collections
dups = [item for item, count in collections.Counter([os.path.basename(f) for f in f_list_cloudmask_final]).items() if count > 1]
print(len(dups))
dups[0:10]

9


['WV02_20130617_M1BS_1030010024CC3100-toa.cloudmask.tif',
 'WV02_20121005_M1BS_103001001BA83B00-toa.cloudmask.tif',
 'WV02_20190620_M1BS_1030010092635900-toa.cloudmask.tif',
 'WV02_20190620_M1BS_103001009409EB00-toa.cloudmask.tif',
 'WV03_20200715_M1BS_104001005F2B5A00-toa.cloudmask.tif',
 'WV03_20200715_M1BS_104001005E0B2100-toa.cloudmask.tif',
 'WV02_20200506_M1BS_10300100A622DE00-toa.cloudmask.tif',
 'WV02_20190619_M1BS_103001009367EC00-toa.cloudmask.tif',
 'WV02_20190619_M1BS_103001009561CF00-toa.cloudmask.tif']

In [24]:
# Get missing
list_footprints_cloudmask_finished = [os.path.basename(f).split('-toa.cloudmask')[0] for f in glob.glob(os.path.join(OUTDIR, 'tmp') + '/*.gpkg' )]
print(f'{len(list_footprints_cloudmask_finished)} that have been footprinted.')

#list_footprints_cloudmask_missing = [stripname for stripname in stripname_list if stripname not in list_footprints_cloudmask_finished]
list_footprints_cloudmask_missing = [r_fn for r_fn in f_list_cloudmask_final if os.path.basename(r_fn).split('-toa.cloudmask')[0] not in list_footprints_cloudmask_finished]

print(f'{len(list_footprints_cloudmask_missing)} cloudmasks that need footprinting.')
print(list_footprints_cloudmask_missing[0])

2768 that have been footprinted.
336 cloudmasks that need footprinting.
/explore/nobackup/projects/ilab/data/srlite/products/srlite_1.0.1/cloudmask/alaska_batch_1/split_6/WV02_20170928_M1BS_103001007388B900-toa.cloudmask.tif


### Multiprocess indiv cloudmask GDFs

In [ ]:
if False:
    f_gdf_lst = [footprintlib.footprint_cloudmask(r_fn, CLOUD_VAL, INVERT=False, N_ITER=N_ITER, OUTDIR=os.path.join(OUTDIR, 'tmp')) for r_fn in list_footprints_cloudmask_missing]
else:
    from multiprocessing import Pool
    from functools import partial

    with Pool(processes=30) as pool:
        pool.map(partial(footprintlib.footprint_cloudmask, NEW_NDV=CLOUD_VAL, INVERT=False, N_ITER=N_ITER, OUTDIR=os.path.join(OUTDIR, 'tmp')), list_footprints_cloudmask_missing)

### Build a single GDF of all indiv cloudmask footprints for SRLite

In [26]:
with Pool(processes=30) as pool:
    f_gdf_lst = pool.map(partial(gpd.read_file), [ footprint_fn for footprint_fn in glob.glob(os.path.join(OUTDIR, 'tmp')+'/*gpkg')] )
    #f_gdf_lst = [ gpd.read_file(footprint_fn) for footprint_fn in glob.glob(os.path.join(OUTDIR, 'tmp')+'/*gpkg')]

In [30]:
footprint_gdf_cloudmask = footprintlib.build_footprint_db(f_gdf_lst, TO_GCS=True, WRITE_GPKG=False, OUT_F_NAME='', OUT_LYR_NAME=TARGET, DROP_DUPLICATES=True)

Building GDF from list...
Converting to each to GCS...


In [34]:
nowtime = pd.Timestamp.now().strftime('%Y%m%d')
print(f"Current time:\t{nowtime}")

Current time:	20230720


In [35]:
out_gdf_SR_CLOUDMASK_fn = f'{OUTDIR}/footprints_vhr_cloudmask_alaska_{nowtime}.gpkg'
footprint_gdf_cloudmask.to_file(out_gdf_SR_CLOUDMASK_fn, driver='GPKG')
print(out_gdf_SR_CLOUDMASK_fn)

/explore/nobackup/people/pmontesa/srlite/footprints_vhr_cloudmask_alaska_20230720.gpkg
